In [1]:
import os, re, json
from unidecode import unidecode
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
PIANO_SOLO_TSV = '220214_debussy_piano_solo.tsv'
SEARCH_JSON = 'spotify_search_results.json'
RESULT_JSON = 'spotify_tracks_data.json'

In [2]:
def make_keys(lesure_col, mvt_col):
    lesure_format =  '{:03d}'.format
    mvt_format =  '{:02d}'.format
    isna = lesure_col.isna()
    lesure_col = lesure_col.apply(lesure_format)
    mvt_col = mvt_col.copy()
    mvt_col[mvt_col.notna()] = mvt_col.apply(mvt_format)
    key = lesure_col + ('-' + mvt_col.astype('string')).fillna('')
    key = key.where(~isna, pd.NA)
    return key
    
client_credentials_manager = SpotifyClientCredentials('5ee5f9cba29f454fabd7bb5316eff69e','0ebdedebbfd74122b9ab666ad6f346eb')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
pieces = pd.read_csv(PIANO_SOLO_TSV, sep='\t')
pieces = pieces[pieces.title.notna()]
lesure_regex = r"^L ?(?P<lesure>\d{1,3})(?:\/(?P<mvt>\d{1,2}))?$"
pieces = pd.concat([pieces.L.str.extract(lesure_regex, expand=True).astype('Int64'), pieces.drop(columns='L')], axis=1)
key_col = make_keys(pieces.lesure, pieces.mvt)
key_col.loc[key_col.isna()] = pieces.title.str.lower().apply(unidecode)
pieces.insert(2, 'key', key_col)
pieces.head(10)

,lesure,mvt,key,year,title,availability,comments,pdf,source
0,9,<NA>,009,1880,Danse bohémienne,True,to check,NaN,https://musescore.com/yazen23434/danse-boh-mie...
1,<NA>,<NA>,intermede,1880,Intermède,False,NaN,NaN,NaN
2,66,1,066-1,1888,Arabesque I: Andantino con moto in E Major,True,NaN,https://imslp.org/wiki/Special:ReverseLookup/2821,https://musescore.com/hmscomp/debussy-premiere...
3,67,<NA>,067,1890,Mazurka,True,several errors,NaN,https://musescore.com/user/4887176/scores/5699433
4,68,<NA>,068,1890,Rêverie,True,NaN,NaN,https://musescore.com/hmscomp/r-verie-claude-d...
5,69,<NA>,069,1890,Tarentelle styrienne,True,NaN,NaN,https://musescore.com/user/56409/scores/263276...
6,70,<NA>,070,1890,Ballade slave,True,NaN,NaN,https://musescore.com/user/3797871/scores/5825949
7,71,<NA>,071,1890,Valse romantique,True,few notes to fix,NaN,https://musescore.com/user/4887176/scores/5696...
8,75,1,075-1,1890-1905,Prélude,True,NaN,NaN,https://musescore.com/user/85337/scores/615026...
9,75,2,075-2,1890-1905,Menuet,True,few notes to fix(free version),NaN,https://musescore.com/user/85337/scores/614817...


In [3]:
if os.path.isfile(SEARCH_JSON):
    with open(SEARCH_JSON, 'r', encoding='utf-8') as f:
        searches = json.load(f)
        print("Loaded search results from " + SEARCH_JSON)
else:
    searches = {}
    for key, lesure, title in pieces[['key', 'lesure', 'title']].itertuples(index=False):
        query = f"debussy {unidecode(title)}"
        if not pd.isnull(lesure):
            query += f" {lesure}"
        print(f"{key}: query string '{query}'")
        searches[key] = sp.search(query, limit=50, type='track')
    with open(SEARCH_JSON, 'w', encoding='utf-8') as f:
        json.dump(searches, f)
        print("Search results stored as " + SEARCH_JSON)

009: query string 'debussy Danse bohemienne 9'
intermede: query string 'debussy Intermede'
066-1: query string 'debussy Arabesque I: Andantino con moto in E Major 66'
067: query string 'debussy Mazurka 67'
068: query string 'debussy Reverie 68'
069: query string 'debussy Tarentelle styrienne 69'
070: query string 'debussy Ballade slave 70'
071: query string 'debussy Valse romantique 71'
075-1: query string 'debussy Prelude 75'
075-2: query string 'debussy Menuet 75'
075-3: query string 'debussy Clair de lune 75'
075-4: query string 'debussy Passepied 75'
066-2: query string 'debussy Arabesque II: Allegretto scherzando in G Major 66'
082: query string 'debussy Nocturne 82'
087-1: query string 'debussy Lent, doux et melancolique 87'
087-2: query string 'debussy Souvenir du Louvre 87'
087-3: query string 'debussy Quelques aspects de 'Nous n'irons plus au bois" 87'
095-2: query string 'debussy Sarabande 95'
095-1: query string 'debussy Prelude 95'
095-3: query string 'debussy Toccata 95'
0

In [4]:
{key: len(search['tracks']['items']) for key, search in searches.items() if len(search['tracks']['items']) < 5}

{'123-9': 2}

### Obtaining dict structure 
leisure -> [{artist, id, duration_ms, release_date}]

In [5]:
pieces_data = {}
for key, search_results in searches.items():
    tracks = search_results['tracks']['items']
    if not tracks: print(f'{"missing tracks for "}{key}')
    search_tracks = []    
    already_found = []
    for (idx, track) in enumerate(tracks):
        artists = [artist['name'] for artist in track['artists'] if 'debussy' not in artist['name'].lower()]
        n_artists = len(artists)
        if n_artists == 0:
            print(f"{key}: track[{idx}] doesn't come with artist's name, skipping")
            continue
        if n_artists > 1:
            print(f"{key}: track[{idx}] has more than 1 artist: {artists}, skipping")
            continue
        artist = artists[0]
        duration_ms = track['duration_ms']
        id_tuple = (artist, duration_ms)
        if id_tuple in already_found:
            continue
        already_found.append(id_tuple)
        name = track['name']
        id = track['id']
        release_date = track['album']['release_date']        
        
        track_info = {
            'name' : name,
            'artist' : artist,
            'id' : id,
            'duration_ms' : duration_ms, 
            'release_date' : release_date  
        }
        search_tracks.append(track_info)
    pieces_data[key] = search_tracks
pieces_data['123-9']

intermede: track[0] has more than 1 artist: ['Michael Barenboim', 'Daniel Barenboim'], skipping
intermede: track[1] has more than 1 artist: ['Rafał Blechacz', 'Bomsori'], skipping
intermede: track[2] has more than 1 artist: ['Michael Barenboim', 'Daniel Barenboim'], skipping
intermede: track[3] has more than 1 artist: ['Michael Barenboim', 'Daniel Barenboim'], skipping
intermede: track[4] has more than 1 artist: ['David Oistrakh', 'Frida Bauer'], skipping
intermede: track[5] has more than 1 artist: ['Rafal Blechacz', 'Bomsori'], skipping
intermede: track[6] has more than 1 artist: ['David Oistrakh', 'Frida Bauer'], skipping
intermede: track[7] has more than 1 artist: ['Augustin Dumay', 'Maria João Pires'], skipping
intermede: track[8] has more than 1 artist: ['Rafał Blechacz', 'Bomsori'], skipping
intermede: track[9] has more than 1 artist: ['Augustin Dumay', 'Maria João Pires'], skipping
intermede: track[10] has more than 1 artist: ['Martha Argerich', 'Dora Schwarzberg'], skipping
int

[{'name': 'Préludes, Livre II, L. 123: No. 9, Hommage à Samuel Pickwick esq. P.P.M.P.C.',
  'artist': 'Jérome Granjon',
  'id': '4ag3EZtqO8MiMWlo5vKIKQ',
  'duration_ms': 154932,
  'release_date': '2011'}]

### Output track names to exclude wrong ones

In [6]:
track_titles = {}
for key, title in pieces.loc[pieces.key.notna(), ['key', 'title']].itertuples(index=False):
    recordings = pieces_data[key]
    track_info = [(track['name'], track['artist']) for track in recordings]
    track_titles[(key, title)] = list(enumerate(track_info))
track_titles

{('009',
  'Danse bohémienne'): [(0,
   ('Danse bohémienne, L.9', 'Gordon Fergus-Thompson')), (1,
   ('Danse bohémienne, L. 9', 'Werner Haas')), (2,
   ('Danse bohémienne, L. 9, CD 4', 'Jean-Efflam Bavouzet')), (3,
   ('Debussy: Danse bohémienne, CD 4, L. 9', 'Aldo Ciccolini')), (4,
   ('Danse Bohémienne, L.9', 'Jean-Yves Thibaudet')), (5,
   ('Danse bohémienne, L. 9', 'Bruno Canino')), (6,
   ('Danse Bohémienne, L. 9', 'John Paul Jalwan')), (7,
   ('Debussy: Danse bohémienne, CD 4, L. 9', 'Walter Gieseking')), (8,
   ('Debussy: Danse bohémienne, CD 4, L. 9', 'Walter Gieseking')), (9,
   ('Debussy: Danse bohémienne, CD 4, L. 9', 'Aldo Ciccolini')), (10,
   ('Debussy: Danse bohémienne, CD 4, L. 9', 'Monique Haas')), (11,
   ('Claude Debussy: Danse bohémienne, L 9 (1880)',
    'Jean-Pierre Armengaud')), (12,
   ('Danse Bohémienne, L.9', 'Jean-Yves Thibaudet')), (13,
   ('Danse bohémienne, L.9', 'Jean-Louis Haguenauer')), (14,
   ('Danse Bohémienne, L. 9', 'Daniel Ericourt')), (15,
   ('D

#### Exclude wrong search results and store

In [7]:
pieces_data['095-1'] = pieces_data['095-1'][:21] + pieces_data['095-1'][23:]
with open(RESULT_JSON, "w", encoding='utf-8') as f:
    json.dump(pieces_data, f)    

### Function for computing median 
returns {leisure->median_dur}

In [8]:

def compute_median_from_tracks_list(filename):
    
    # reading from file 
    pieces_data = json.loads(open(filename, "r").read())

    # obtaining medians 
    median_durations = {}
    for piece in pieces_data:
        durations = [t['duration_ms'] for t in  pieces_data[piece]] 
        if not durations: 
            print(piece + " duration missing")
            median_dur = -1
        else: 
            median_dur = pd.DataFrame(durations).median().values[0]
        median_durations[piece] = median_dur
    return median_durations

print(compute_median_from_tracks_list(RESULT_JSON))
    
    

131-2 duration missing
{'009': 124599.5, 'intermede': 256640.0, '066-1': 241860.5, '067': 175913.0, '068': 265426.5, '069': 331529.0, '070': 396014.0, '071': 221450.0, '075-1': 245829.0, '075-2': 249453.0, '075-3': 303454.0, '075-4': 223143.0, '066-2': 213933.0, '082': 406573.0, '087-1': 223646.0, '087-2': 291493.0, '087-3': 242945.0, '095-2': 311086.5, '095-1': 246743.0, '095-3': 233760.0, '099': 294606.5, '100-1': 300173.0, '100-2': 322066.0, '100-3': 226853.0, '105': 283240.0, '106': 355413.0, '108': 54760.0, '110-1': 306487.5, '110-2': 420733.0, '110-3': 207957.0, '113-1': 133640.0, '113-2': 207704.5, '113-3': 165133.0, '113-4': 157740.0, '113-5': 144386.0, '113-6': 175715.0, '111-1': 271569.0, '111-2': 332779.5, '111-3': 246329.5, '114': 97448.0, '115': 141874.0, '117-1': 197746.0, '117-10': 398693.0, '117-2': 233020.0, '117-3': 132120.0, '117-5': 186133.0, '117-6': 257732.0, '117-7': 211000.0, '117-9': 163066.0, '117-11': 167260.0, '117-12': 147076.0, '117-4': 218780.0, '117-8': 

In [4]:
links = {} # collected from the search 'mozart complete piano'
links['say'] = 'spotify:album:16RpGAof6TMerrEaGfG1sL'
links['barenboim'] = 'spotify:album:0RLPXICOApN0gXaqutndwJ'
links['brautigam'] = 'spotify:album:1oOpxTQAQ7YB8oqOwnnzq5'
links['leygraf'] = 'spotify:album:3EL4qbNiOlhOgjVDkTU499'
links['gieseking'] = 'spotify:album:3ADG19bG9seOkh5HSTDnwo'
#links['van_oort'] = 'spotify:album:0DQAS2R5f4eYIJwns0mxCL' (deviating #movements in K. 331)
links['endres'] = 'spotify:album:4lIEN4Ll8PIkgE30KGFyCj'
#links['uchida'] = 'spotify:album:7DrgRyCKnviHUTlTyP32wA' (movement numbers not Roman)
links['mauser'] = 'spotify:album:5N0Edf6DsZQ56Q19vKqEOg'
links['deyanova'] = 'spotify:album:3U8EtwYB2NPnGtlfyFw5z7'
links['mamou'] = 'spotify:album:0yAYm26SkzQtil5SZvxkOu'
#links['badura-skoda'] = 'spotify:album:6uiVpRRqZQOY3lc6GhkZux' (includes movement numbers IV. and i.)
#links['tirimo'] = 'spotify:album:1tTqsxhN5BFD4sSR3xQ8hz' (no movement numbers)
kv = ['279','280','281','282','283','284','309','311','310','330','331','332','333','457','533','545','570','576']

In [5]:
a = sp.album('spotify:album:16RpGAof6TMerrEaGfG1sL')
a.keys()

dict_keys(['album_type', 'artists', 'available_markets', 'copyrights', 'external_ids', 'external_urls', 'genres', 'href', 'id', 'images', 'label', 'name', 'popularity', 'release_date', 'release_date_precision', 'total_tracks', 'tracks', 'type', 'uri'])

In [6]:
selected = {k: v for i, (k, v) in enumerate(links.items()) if i in [0,1,5,6,7,8]}
selected

{'say': 'spotify:album:16RpGAof6TMerrEaGfG1sL',
 'barenboim': 'spotify:album:0RLPXICOApN0gXaqutndwJ',
 'endres': 'spotify:album:4lIEN4Ll8PIkgE30KGFyCj',
 'mauser': 'spotify:album:5N0Edf6DsZQ56Q19vKqEOg',
 'deyanova': 'spotify:album:3U8EtwYB2NPnGtlfyFw5z7',
 'mamou': 'spotify:album:0yAYm26SkzQtil5SZvxkOu'}

In [7]:
for name, id in selected.items():
    a = sp.album(id)
    print(f"{a['artists'][1]['name']} - {a['label']} - {a['release_date']}")

Fazıl Say - Warner Classics - 2016-09-30
Daniel Barenboim - Warner Classics - 2013-08-02
Michael Endres - Oehms Classics - 2011-03-01
Siegfried Mauser - Celestial Harmonies - 2014-10-24
Marta Deyanova - Nimbus Records - 1996
Roberte Mamou - Ligia - 2015-10-30


In [8]:
suffixes = {'I':'-1','II':'-2', 'III':'-3'}
data = {('K'+k,r):[] for k in kv for r in ['-1','-2','-3']}
ids = {'id': {}}
for player,uri in links.items():
    tracks = sp.album_tracks(uri)
    while tracks:
        for t in tracks['items']:
            for k in kv:
                if re.search(k+'[^a-z]',t['name']):
                    try:
                        m = re.search(r'\s(I|II|III)\.',t['name'])[1]
                    except:
                        print(player,t['name'])
                    key = ('K'+k,suffixes[m])
                    data[key].append(t['duration_ms'])
                    ids['id'][(f"K{k}{suffixes[m]}", player)] = t['id']
        tracks = sp.next(tracks)
df = pd.DataFrame.from_dict(data,orient='index',columns=links.keys())
#df.index = pd.MultiIndex.from_tuples(df.index,names=['K','movement'])
df.index = [''.join(i) for i in df.index]

In [9]:
#features = sp.audio_features(list(track_ids.id.values))

In [10]:
track_ids = pd.DataFrame(ids)
features = track_ids.id.map(lambda ID: sp.audio_features(ID)[0]['tempo'])

In [ ]:
features.unstack().iloc[:, [0,1,5,6,7,8]].to_csv('spotify_bpm.tsv')

In [ ]:
df = df / 1000

In [ ]:
df.T.describe()

,K279-1,K279-2,K279-3,K280-1,K280-2,K280-3,K281-1,K281-2,K281-3,K282-1,K282-2,K282-3,K283-1,K283-2,K283-3,K284-1,K284-2,K284-3,K309-1,K309-2,K309-3,K311-1,K311-2,K311-3,K310-1,K310-2,K310-3,K330-1,K330-2,K330-3,K331-1,K331-2,K331-3,K332-1,K332-2,K332-3,K333-1,K333-2,K333-3,K457-1,K457-2,K457-3,K533-1,K533-2,K533-3,K545-1,K545-2,K545-3,K570-1,K570-2,K570-3,K576-1,K576-2,K576-3
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.00000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.0000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,323.900333,375.919556,231.232333,304.825000,371.573000,204.070778,301.687111,335.740444,268.962444,348.347667,196.347778,163.340444,265.084111,335.420333,277.295889,340.278222,278.356667,955.058778,386.208444,377.078111,365.925444,309.904778,344.118111,337.881111,393.490111,546.297333,185.774444,422.44400,390.893000,358.315333,733.158222,350.436778,215.008667,433.098889,285.482667,455.645556,466.556667,514.192444,376.479667,406.587778,451.963889,289.743333,495.368444,609.790778,367.770111,211.001111,333.134444,105.3330,377.522667,483.989222,216.691444,290.857556,321.891667,260.042556
std,68.759471,104.649948,51.977281,61.355322,106.249829,44.517995,66.823153,81.200070,27.758446,95.377074,82.379816,34.817696,50.496798,70.151007,62.828702,88.926275,53.519133,127.110890,99.194778,44.232440,28.902464,73.809821,30.882755,127.045860,101.387638,85.387552,10.095433,97.26903,79.798608,72.263346,150.799063,52.770283,17.733365,104.768013,33.347504,124.644551,115.744015,108.835705,28.080755,108.602033,41.984264,16.270506,101.204457,147.798038,29.417508,35.454730,43.287811,6.9794,64.793457,79.813447,23.037094,30.099450,25.948377,16.145462
min,220.466000,210.653000,166.373000,219.293000,217.880000,140.800000,202.040000,219.800000,208.173000,186.733000,1.493000,105.973000,181.893000,221.053000,196.693000,221.826000,200.666000,712.146000,241.306000,314.573000,318.000000,209.000000,292.893000,1.493000,257.000000,409.733000,166.000000,289.45300,228.666000,250.533000,447.213000,253.773000,194.160000,272.853000,236.653000,270.293000,297.680000,346.000000,319.000000,228.453000,367.706000,266.933000,357.253000,420.826000,307.000000,179.586000,255.706000,90.3200,277.453000,330.746000,162.906000,216.706000,291.120000,233.093000
25%,284.560000,301.826000,196.240000,265.840000,301.000000,179.040000,262.840000,301.160000,257.399000,287.666000,188.000000,139.999000,241.000000,292.666000,235.506000,305.333000,252.346000,904.826000,355.546000,354.666000,348.120000,272.986000,328.506000,364.320000,327.666000,485.506000,180.360000,372.00000,350.000000,315.653000,677.000000,325.080000,204.426000,385.333000,274.320000,403.666000,421.666000,425.000000,366.213000,374.720000,452.826000,280.279000,452.066000,491.000000,363.946000,186.373000,294.640000,103.0400,336.066000,448.666000,213.146000,288.333000,306.440000,244.173000
50%,301.506000,387.226000,204.759000,283.026000,402.026000,184.826000,278.012000,343.493000,273.653000,369.039000,221.466000,180.000000,247.493000,333.893000,260.026000,314.866000,259.200000,949.320000,362.706000,368.306000,365.426000,277.212000,335.733000,378.826000,376.866000,533.160000,188.333000,380.30600,397.240000,343.760000,774.506000,340.533000,208.399000,427.400000,288.306000,430.293000,445.972000,545.960000,382.106000,387.506000,455.000000,285.520000,475.866000,610.666000,366.240000,192.000000,359.226000,107.4660,356.026000,486.000000,224.666000,297.786000,318.000000,267.600000
75%,395.840000,453.053000,273.000000,381.000000,455.760000,247.333000,379.000000,385.093000,289.840000,390.826000,236.133000,185.893000,315.000000,407.160000,349.000000,439.200000,328.733000,1019.306000,

In [ ]:
df.describe()

,say,barenboim,brautigam,leygraf,gieseking,endres,mauser,deyanova,mamou
count,54.000000,54.000000,54.000000,54.000000,54.00000,54.000000,54.000000,54.000000,54.000000
mean,404.868315,372.965296,411.140000,278.242389,292.60237,334.416407,371.853833,395.629000,362.868833
std,165.827070,169.420237,162.787646,110.225199,129.81118,132.334877,166.410524,152.062887,156.131787
min,90.320000,101.400000,115.200000,103.040000,1.49300,107.826000,108.772000,110.000000,103.973000
25%,304.603250,264.433250,297.360000,209.413250,228.06300,260.833000,277.412000,292.000000,282.719500
50%,374.313000,360.966000,383.220000,259.313000,280.01950,305.166500,357.646000,380.086500,339.540000
75%,463.539500,426.183250,485.550000,325.973250,354.84600,378.619500,413.685500,499.369500,417.500000
max,1019.306000,1004.866000,949.320000,712.146000,910.53300,904.826000,1097.386000,866.173000,1130.973000


In [ ]:
mean_of_means = df.describe().loc['mean'].mean()
mean_of_means

358.2873827160494

In [ ]:
std_of_means = df.describe().loc['mean'].std()
std_of_means

47.52578712570362

In [ ]:
(df.describe().loc['mean'].T - mean_of_means).abs() > std_of_means

say          False
barenboim    False
brautigam     True
leygraf       True
gieseking     True
endres       False
mauser       False
deyanova     False
mamou        False
Name: mean, dtype: bool

In [ ]:
df.iloc[:,[0,1,5,6,7,8]].T.describe()

,K279-1,K279-2,K279-3,K280-1,K280-2,K280-3,K281-1,K281-2,K281-3,K282-1,K282-2,K282-3,K283-1,K283-2,K283-3,K284-1,K284-2,K284-3,K309-1,K309-2,K309-3,K311-1,K311-2,K311-3,K310-1,K310-2,K310-3,K330-1,K330-2,K330-3,K331-1,K331-2,K331-3,K332-1,K332-2,K332-3,K333-1,K333-2,K333-3,K457-1,K457-2,K457-3,K533-1,K533-2,K533-3,K545-1,K545-2,K545-3,K570-1,K570-2,K570-3,K576-1,K576-2,K576-3
count,6.000000,6.000000,6.000000,6.00000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000
mean,332.799500,402.726167,237.288667,312.66200,402.248500,208.099500,308.479667,352.968500,259.717167,380.168333,228.406167,161.986333,271.135167,342.779500,286.952833,362.270833,298.833000,1003.921667,411.575000,388.801833,368.532833,319.306167,356.026167,378.666167,421.655167,585.763833,185.639500,433.359500,411.693000,366.548667,788.770833,372.424167,208.646333,464.201833,285.324000,484.830667,497.088333,540.862000,375.835167,445.568500,447.334833,292.641833,518.483833,634.395167,362.219667,208.844000,346.957333,103.715167,404.844167,508.561833,221.930667,303.619833,325.728667,259.575000
std,58.502479,79.106209,40.925495,54.60085,79.797372,37.457023,60.277567,47.772182,29.433897,63.186874,33.554739,28.438450,36.315301,64.522193,56.945136,69.144966,51.612648,103.836254,71.000364,49.299291,35.543225,70.540190,28.088014,14.370736,84.731542,67.192696,12.198812,93.266237,64.153863,59.534984,80.462374,43.109195,12.431374,69.459973,41.699032,96.470047,86.818058,65.193706,32.196473,75.863658,51.878956,19.342672,74.516429,112.421781,32.611373,35.719689,32.493723,7.305157,58.027770,68.998357,12.565781,12.215910,31.493255,12.712009
min,264.773000,301.826000,196.240000,264.40000,301.000000,178.826000,261.066000,301.160000,208.173000,287.666000,188.000000,127.760000,241.000000,264.012000,235.506000,305.333000,215.840000,866.173000,355.546000,314.573000,318.000000,264.533000,324.253000,360.533000,327.666000,485.506000,166.000000,353.666000,334.000000,303.000000,677.000000,322.000000,194.160000,385.333000,236.653000,403.666000,421.666000,425.000000,319.000000,374.720000,367.706000,266.933000,452.066000,491.000000,307.000000,179.586000,283.666000,90.320000,336.066000,423.826000,201.000000,288.333000,291.120000,243.333000
25%,296.635500,350.306500,201.939750,275.63625,343.636250,181.806500,266.590000,320.492750,251.349500,362.879250,210.746250,140.832500,244.172750,295.516000,246.986500,312.256000,270.283250,929.836000,357.426000,359.832750,348.856250,274.803000,331.649500,371.635750,349.466250,543.616000,180.182750,373.100000,361.810000,332.510000,728.376500,341.469750,201.026250,421.109750,263.976250,424.473000,439.672750,529.700000,369.159750,382.016000,417.836250,281.589250,473.096250,573.325750,348.356500,182.722750,346.306500,102.043250,359.883000,457.999500,216.026000,298.659500,298.500000,248.466250
50%,315.173000,401.239000,233.546000,287.01300,409.446000,188.912500,277.926000,344.366000,264.085500,378.999500,226.066000,161.666500,254.259500,336.999500,263.179000,328.613000,325.433000,1012.086000,377.606000,397.166500,372.039000,279.499000,356.540000,378.999500,419.053000,597.733000,188.392500,384.966000,409.253000,346.613000,809.539500,366.033000,206.412500,441.306000,281.606000,432.926000,453.712500,550.480000,380.053000,432.953000,461.846000,292.593000,480.166000,611.833000,372.226000,191.719500,359.552500,105.899500,400.773000,503.220000,225.799000,302.640000,324.966500,264.526000
75%,379.090000,443.602750,270.333250,358.50000,446.036500,233.749500,353.753000,375.129500,279.782500,390.359500,234.766250,182.499750,301.506500,401.483250,328.333000,417.840000,331.562750,1077.8

In [ ]:
from scipy.stats import hmean
comparison = df.iloc[:,[0,1,5,6,7,8]].T.describe().loc[['mean','50%']]
comparison.append(pd.Series(df.iloc[:,[0,1,5,6,7,8]].T.apply(hmean),name='hmean'))

<ipython-input-17-749a49352309>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comparison.append(pd.Series(df.iloc[:,[0,1,5,6,7,8]].T.apply(hmean),name='hmean'))


,K279-1,K279-2,K279-3,K280-1,K280-2,K280-3,K281-1,K281-2,K281-3,K282-1,K282-2,K282-3,K283-1,K283-2,K283-3,K284-1,K284-2,K284-3,K309-1,K309-2,K309-3,K311-1,K311-2,K311-3,K310-1,K310-2,K310-3,K330-1,K330-2,K330-3,K331-1,K331-2,K331-3,K332-1,K332-2,K332-3,K333-1,K333-2,K333-3,K457-1,K457-2,K457-3,K533-1,K533-2,K533-3,K545-1,K545-2,K545-3,K570-1,K570-2,K570-3,K576-1,K576-2,K576-3
mean,332.799500,402.726167,237.288667,312.662000,402.248500,208.099500,308.479667,352.96850,259.717167,380.168333,228.406167,161.986333,271.135167,342.779500,286.952833,362.270833,298.833000,1003.921667,411.575000,388.801833,368.532833,319.306167,356.026167,378.666167,421.655167,585.763833,185.639500,433.359500,411.693000,366.548667,788.770833,372.424167,208.646333,464.201833,285.324000,484.830667,497.088333,540.862000,375.835167,445.568500,447.334833,292.641833,518.483833,634.395167,362.219667,208.844000,346.957333,103.715167,404.844167,508.561833,221.930667,303.619833,325.728667,259.575000
50%,315.173000,401.239000,233.546000,287.013000,409.446000,188.912500,277.926000,344.36600,264.085500,378.999500,226.066000,161.666500,254.259500,336.999500,263.179000,328.613000,325.433000,1012.086000,377.606000,397.166500,372.039000,279.499000,356.540000,378.999500,419.053000,597.733000,188.392500,384.966000,409.253000,346.613000,809.539500,366.033000,206.412500,441.306000,281.606000,432.926000,453.712500,550.480000,380.053000,432.953000,461.846000,292.593000,480.166000,611.833000,372.226000,191.719500,359.552500,105.899500,400.773000,503.220000,225.799000,302.640000,324.966500,264.526000
hmean,324.594831,389.888182,231.469976,305.411896,388.746739,203.055542,299.700780,347.87998,256.640763,371.413946,224.585314,157.809910,267.353219,332.609609,278.433403,352.309702,290.222712,994.801060,402.225507,383.303703,365.655529,307.984304,354.174044,378.220372,407.525913,579.013034,184.947176,418.513676,403.326105,359.166288,781.672013,368.361709,208.046237,455.913775,280.670625,470.543072,485.590996,533.544926,373.362341,435.295837,442.024774,291.587813,510.265890,618.199275,359.593618,204.192917,344.048180,103.252076,397.945331,500.807296,221.311911,303.220834,323.202887,259.043897


## Store median durations to file

In [ ]:
comparison.T.to_csv('performance_durations.tsv', sep='\t')

In [ ]:
import cufflinks as cf
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)
cf.go_offline()

In [ ]:
seconds = df.copy()

In [ ]:
seconds.T.iplot(kind='box')

In [ ]:
seconds.iloc[:,[0,1,5,6,7,8]].iplot(mode='markers')

In [ ]:
seconds.iloc[:,[0,1,5,6,7,8]].T.iplot(kind='box')

In [ ]:
df.loc['K533-1'].iloc[[0,1,5,6,7,8]]

say          625.506
barenboim    452.066
endres       472.173
mauser       484.466
deyanova     600.826
mamou        475.866
Name: K533-1, dtype: float64

In [ ]:
track_ids.loc[('K533-1', 'gieseking')]

id    3YkrwCvnfY7mJrb1QXQRy5
Name: (K533-1, gieseking), dtype: object

In [ ]:
median_shift = seconds.T.median() - seconds.iloc[:,[0,1,5,6,7,8]].T.median()
median_shift

K279-1   -13.6670
K279-2   -14.0130
K279-3   -28.7870
K280-1    -3.9870
K280-2    -7.4200
K280-3    -4.0865
K281-1     0.0860
K281-2    -0.8730
K281-3     9.5675
K282-1    -9.9605
K282-2    -4.6000
K282-3    18.3335
K283-1    -6.7665
K283-2    -3.1065
K283-3    -3.1530
K284-1   -13.7470
K284-2   -66.2330
K284-3   -62.7660
K309-1   -14.9000
K309-2   -28.8605
K309-3    -6.6130
K311-1    -2.2870
K311-2   -20.8070
K311-3    -0.1735
K310-1   -42.1870
K310-2   -64.5730
K310-3    -0.0595
K330-1    -4.6600
K330-2   -12.0130
K330-3    -2.8530
K331-1   -35.0335
K331-2   -25.5000
K331-3     1.9865
K332-1   -13.9060
K332-2     6.7000
K332-3    -2.6330
K333-1    -7.7405
K333-2    -4.5200
K333-3     2.0530
K457-1   -45.4470
K457-2    -6.8460
K457-3    -7.0730
K533-1    -4.3000
K533-2    -1.1670
K533-3    -5.9860
K545-1     0.2805
K545-2    -0.3265
K545-3     1.5665
K570-1   -44.7470
K570-2   -17.2200
K570-3    -1.1330
K576-1    -4.8540
K576-2    -6.9665
K576-3     3.0740
dtype: float64

In [ ]:
median_shift.mean()

-11.757481481481486

In [ ]:
mean_shift = seconds.T.mean() - seconds.iloc[:,[0,1,5,6,7,8]].T.mean()
mean_shift

K279-1    -8.899167
K279-2   -26.806611
K279-3    -6.056333
K280-1    -7.837000
K280-2   -30.675500
K280-3    -4.028722
K281-1    -6.792556
K281-2   -17.228056
K281-3     9.245278
K282-1   -31.820667
K282-2   -32.058389
K282-3     1.354111
K283-1    -6.051056
K283-2    -7.359167
K283-3    -9.656944
K284-1   -21.992611
K284-2   -20.476333
K284-3   -48.862889
K309-1   -25.366556
K309-2   -11.723722
K309-3    -2.607389
K311-1    -9.401389
K311-2   -11.908056
K311-3   -40.785056
K310-1   -28.165056
K310-2   -39.466500
K310-3     0.134944
K330-1   -10.915500
K330-2   -20.800000
K330-3    -8.233333
K331-1   -55.612611
K331-2   -21.987389
K331-3     6.362333
K332-1   -31.102944
K332-2     0.158667
K332-3   -29.185111
K333-1   -30.531667
K333-2   -26.669556
K333-3     0.644500
K457-1   -38.980722
K457-2     4.629056
K457-3    -2.898500
K533-1   -23.115389
K533-2   -24.604389
K533-3     5.550444
K545-1     2.157111
K545-2   -13.822889
K545-3     1.617833
K570-1   -27.321500
K570-2   -24.572611


In [ ]:
mean_shift.mean()

-15.479564814814816